<font color=maroon size=5.5>Testing Quadrupole Moment Element Calculations with $R_\lambda$ Contraint</font>

<font color=maroon size=4>Aidan Cloonan</font>

<font color=maroon size=4>March 2021</font>

I will be attempting to make some quadrupole moment calculations using the compiled Cartesian data from get_comoving_cartesian_coordinates.ipynb, constraining the data in the FITS file to all members within calculated $R_\lambda$ values. We find this with:

$R_{\lambda} = 1$ $\rm Mpc$ $\rm h^{-1}$ $(\frac{\lambda}  {100})^{0.2}$,

where $\lambda$ is the richness of the cluster. This formula is from [Rykoff, *et al*. (2014)](https://arxiv.org/pdf/1303.3562.pdf), and from what I understand, it should not be used as a reliable estimate of a cluster radius. However, it'll do for testing purposes right now, until I figure out what to do regarding virial radii.

**4/6 Update:** Right now, I'm having trouble importing the `HaloShape` class from my .py file, so I've included it here. I also have marked out a couple of parts in the code I can immediately think about, and I will try to work on and update these some time today. `###` denotes these comments.

In [33]:
# math, array manipulation, etc.
import numpy as np
import esutil                    # must use desc-stack kernel

# get central galaxy/BCG and cluster member data
import GCRCatalogs as gcr

# astropy
import astropy.io.fits as fits                     # writing to and opening FITS files
from astropy.cosmology import FlatLambdaCDM        # making cosmological calculations
from astropy import units as u                     # units
from astropy.table import Table                    # Table data structure

# for timing loops 
from tqdm import *
import time

# plots
import matplotlib.pyplot as plt
%matplotlib inline

# shape modelling
from shape_model_acloonan import read_fits
from shape_model_acloonan import HaloShape

ImportError: cannot import name 'HaloShape' from 'shape_model_acloonan' (/global/u2/a/acloonan/github_cluster_triaxiality/quadrupole_moment/shape_model_acloonan.py)

**Task**: Figure out how to import this file. Then, start going through the weeds of testing the model.

In [ ]:
test_table = read_fits('../fits_files/halos_matched_cosmoDC2.fits')
# redMaPPer catalog
redM_gcr = gcr.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.5.7')

# list all redMaPPer quantities
redM_quantities = redM_gcr.list_all_quantities()
print("All redMaPPer quantities:\n")
print(redM_quantities)

# these are the quantities that describe the clusters and the central galaxies
cluster_quantities = [q for q in redM_quantities if 'member' not in q]
print("\nCluster quantities:\n")
print(cluster_quantities)

# get cluster data
redM_data = Table(redM_gcr.get_quantities(cluster_quantities))

In [17]:
redM_data['R_lambda'] = (redM_data['richness'] / 100) ** 0.2
redM_data['R_lambda_err'] = (redM_data['R_lambda'] 
                             * 0.2 
                             * redM_data['richness_err']) / redM_data['richness']

In [18]:
redM_data

ra_cen_1,dec_cen_4,dec_cen_1,id_cen_1,dec_cen_2,id_cen_4,p_cen_1,p_cen_4,id_cen_3,redshift_err,p_cen_2,scaleval,id_cen_2,dec_cen_3,richness,ra_cen_2,redshift,p_cen_0,ra_cen_4,redshift_true_cg,ra_cen_0,ra,dec,dec_cen_0,richness_err,p_cen_3,cluster_id,maskfrac,ra_cen_3,id_cen_0,R_c,R_c_err
float64,float64,float64,int64,float64,int64,float32,float32,int64,float32,float32,float32,int64,float64,float32,float64,float32,float32,float64,float32,float64,float64,float64,float64,float32,float32,int32,float32,float64,int64,float32,float32
52.519490104517175,-25.727465056757442,-25.71783277825776,1312932492,-25.716281990601974,1313029113,0.21592115,3.1720174e-07,1312945763,0.0039962004,6.0464543e-05,1.000083,1312932494,-25.72165959330821,193.83392,52.50825021974284,0.5134974,0.7840171,52.51910078836549,0.51072353,52.520436430812914,52.520436430812914,-25.717996124671657,-25.717996124671657,4.184529,9.564124e-07,12,0.0,52.527338368381535,1312945563,1.1415265,0.0049287044
52.67672577064746,-25.652592664965763,-25.631890952477875,1313040937,-25.619499813846655,1313033291,4.5898125e-07,7.864599e-13,1313040733,0.005268136,6.6147105e-12,0.99995136,1313040832,-25.64826766089352,90.55774,52.68325184255043,0.5692477,0.9999995,52.67085239603084,0.5693116,52.67840938259691,52.67840938259691,-25.63184326314064,-25.63184326314064,3.3002403,4.609863e-12,154,0.0,52.704325130875795,1313040661,0.98035896,0.0071455403
53.815297020303866,-26.527209889771267,-26.551915451964234,1312583706,-26.53212501634062,1312581745,0.26972476,1.4195513e-06,1312581707,0.0055036736,0.0036682712,1.0007616,1312581805,-26.55508534360349,72.99927,53.805899585778235,0.29388255,0.726603,53.80783815561447,0.297553,53.81457486617126,53.81457486617126,-26.552357796399047,-26.552357796399047,3.414934,2.5736192e-06,385,0.0,53.82069232081629,1312581703,0.9389959,0.008785319
52.245974196694526,-25.993442017532335,-25.977255459458455,1312509756,-26.002606920683938,1312509609,0.21895306,2.0014295e-05,1312509760,0.003030211,0.0009247641,1.0001549,1312509846,-25.999386905587617,46.928513,52.20329292497875,0.1389136,0.7799704,52.32351370006402,0.13520846,52.24966756483943,52.24966756483943,-25.97730838369064,-25.97730838369064,2.072687,0.00013177193,501,0.0,52.24270859041221,1312509606,0.85958207,0.007593015
53.92952728322835,-25.26481545909912,-25.259841882078486,1314225969,-25.253967331170642,1314225959,0.00054260617,3.5773667e-06,1314225945,0.015179389,7.949381e-06,1.0001003,1314225978,-25.254443792942293,56.142517,53.922883232399116,0.9131491,0.9994414,53.924976133834775,0.90448755,53.92820091867718,53.92820091867718,-25.25995999463154,-25.25995999463154,3.064124,4.4778362e-06,466,0.0,53.936475002239725,1314220475,0.89096034,0.009725297
52.94341068114814,-26.093024339912517,-26.07513152296139,1312872359,-26.083584663785977,1312872414,0.057398543,1.1144596e-06,1312872381,0.0045125145,0.00043741154,0.9999347,1312742742,-26.08960903977043,44.563995,52.92667842345666,0.46766934,0.9421399,52.939987550414294,0.46815017,52.940786054153854,52.940786054153854,-26.077918984346976,-26.077918984346976,2.437981,2.3034776e-05,840,0.0,52.942329124748056,1312872294,0.85073996,0.009308357
54.05594218691781,-26.3604879553036,-26.380582482703904,1313029993,-26.3548060687829,1313259860,4.0653977e-05,1.8641332e-07,1313259908,0.007960019,5.5888813e-06,0.9999543,1313259884,-26.375711825692015,34.816063,54.051165644192,0.5953326,0.99995285,54.03072524049983,0.5816059,54.048892077961746,54.048892077961746,-26.36437651092295,-26.36437651092295,2.4545481,7.0932725e-07,1722,0.0,54.032777413001384,1313133021,0.80975926,0.0114176795
53.61554945739801,-27.046327137165985,-27.061130746144766,1312937870,-27.033275772127407,1312937890,0.006912816,2.1405825e-11,1312937910,0.00501389,0.00020798348,1.0002017,1312937862,-27.038243543483723,29.598864,53.6390207221805,0.4982514,0.9928792,53.60605219256883,0.5000259,53.61812432952921,53.61812432952921,-27.049272624733792,-27.049272624733792,2.0237167,3.294623e-10,1768,0.0,53.627

In [38]:
class HaloShape(object):
    
    def __init__(self, cluster_id = None, id_cen_0 = None, ra_cen=None, dec_cen=None, red_cen=None, x_cen=None, y_cen=None, z_cen=None, Rmax=None, verbose = False):
        
        assert cluster_id is not None, "cluster_id is None"     # cosmoDC2 redMaPPer cluster ID
        assert id_cen_0 is not None, "id_cen_0 is None"         # cosmoDC2 galaxy ID of most likely
                                                                # BCG candidate
        assert ra_cen is not None, "ra_cen is None"             
        assert dec_cen is not None, "dec_cen is None" 
        assert red_cen is not None, "red_cen is None" #redshift
        assert x_cen is not None, "x_cen is None" #x, y, z in Mpc/h
        assert y_cen is not None, "y_cen is None"
        assert z_cen is not None, "z_cen is None" 
        assert Rmax is not None, "Rmax is None" #Virial radius in Mpc/h
        
        self.cluster_id = cluster_id
        self.id_cen_0 = id_cen_0
        self.ra_cen = ra_cen
        self.dec_cen = dec_cen
        self.red_cen = red_cen
        self.x_cen = x_cen
        self.y_cen = y_cen
        self.z_cen = z_cen
        self.Rmax = Rmax
        
        #Some statement if verbose is true
        if verbose:
            print("Verbose")
            
        #Other non-instantiated parameters of the halo_shape object
        self.axis_ratio = np.array([1,1,1])
        self.axis_dir = np.identity(3)
        self.converge = False
        self.ptcl_num = 0
        
        #Choose between high_z and low_z folders
        '''if (red_cen >= 0.34) & (red_cen < 0.90):
            self.basepath = setup.buzzard_particles_highz_dir()
        elif (red_cen < 0.34):
            self.basepath = setup.buzzard_particles_lowz_dir()
        else:
            raise Exception('redshift z={} outside of Buzzard range'.format(red_cen))'''
        return 
     
    """
    Iteratively solves the axis ratios and direction until convergence criterion for 
    envelope and shape of halo particles inside the envelope is met. 
    
    #Convergence criterion for envelope same as for particles inside envelope. 
    """
    def evolve(self):
        ptcl_coord = self.read_halo_ptcl()
        self.ptcl_num = len(ptcl_coord[0])
	#too few particles proabably won't converge and may generate error. 
        if self.ptcl_num  < 100:
	    #print "Initial number of particles is ", len(ptcl_coord[0]) 
            return 
        
        #Converge criterion for the envelope
        a = self.axis_ratio[0]; b = self.axis_ratio[1]; c = self.axis_ratio[2]
        q_prev = c/a; s_prev = b/a
        
        conv_err = 1e-6
        conv_iter_max = 100
        env_converge=False
        ptcl_converge=True
        conv_iter=0
        while (ptcl_converge) & (not env_converge) & (conv_iter < conv_iter_max):
            #Performs quad_moment only if there are enough members
            
            ### What is an appropriate minimum number of members?
            
            if self.ptcl_num > 100:
                self.ptcl_num, ptcl_converge, self.axis_ratio, self.axis_dir = self.quad_moment(ptcl_coord)
                #print self.axis_ratio, self.axis_dir            
                a = self.axis_ratio[0]; b = self.axis_ratio[1]; c = self.axis_ratio[2]
                q_cur = c/a; s_cur = b/a

                conv_s = np.abs(1 - s_cur/s_prev); conv_q = np.abs(1 - q_cur/q_prev)
                env_converge = (conv_s < conv_err) & (conv_q < conv_err)
    
                q_prev = q_cur; s_prev = s_cur
                conv_iter += 1

            else:
                self.converge = False
        return


        
        self.converge = ptcl_converge & env_converge & (conv_iter < conv_iter_max)
        return   
        
    """
    Returns the number of DM particles inside halo
    """
    def get_ptcl_num(self):
        return self.ptcl_num
        
    """
    Returns whether shape of halo converge. Need for the envelope and particle inside envelope both to converge
    """
    def get_converge(self):
        return self.converge
    
    """
    Returns array of axis ratio a,b,c -- major, intermediate, minor axis
    """
    def get_axis_ratio(self):
        return self.axis_ratio
    
    """
    Returns axis direction lx, ly, lz -- minor, intermeidate, major axis (different order from a,b,c)
    """
    def get_axis_dir(self):
        return self.axis_dir
    
        
    """
    Reads out the particle positions of those that belong within a given distance to the halo center.
    Inputs halo position and reads halo particle files adjacent to the halo. Calls read_radial_bin() and 
    query_file() to read halo particles. 
    
    Inputs:
    ra_cen, de_cen: RA, DEC of halo center
    red_cen: redshift 
    x_cen, y_cen, z_cen: Unrotated position of halo in Mpc/h
    Rmax: max radius to enclose particles in Mpc/h. 
    
    Returns:
    (3,n) list of particle positions
    """
    def read_halo_ptcl(self):
        r_search = 2*self.Rmax #Search within twice virial radius, large enough to include all potential particles. 
        chi_cen = np.sqrt(self.x_cen**2 + self.y_cen**2 + self.z_cen**2)
        ang = (r_search/chi_cen) * (180./np.pi) # angle to search particles (in degree)

        x = [] # bad practice
        y = [] # bad practice!
        z = [] # bad practice!!
        filename_exists = []

        #Change this part to vary the radius of the envelope. 
        for ra_pm in [-ang,0,ang]: # some particle might be in another patch
            for dec_pm in [-ang,0,ang]:
                for chi_pm in [-r_search,0,r_search]:
                    
                    ### Think about this. This appears to call a file with halo
                    ### particles. How can I call my member table file and query
                    ### a cluster's members here?
                    
                    filename = query_file(self.basepath, ra=self.ra_cen+ra_pm, dec=self.dec_cen+dec_pm, r=chi_cen+chi_pm)
                    if filename in filename_exists: 
                        #print('used')
                        pass
                    else:
                        filename_exists.append(filename)
                        hdr, idx, pos = read_radial_bin(filename, read_pos=True)
                        
                        #pos relative to center less than r_search
                        npart = len(pos)//3
                        pos=pos.reshape(npart, 3) 
                        dist_x = pos[:,0]-self.x_cen; dist_y = pos[:,1]-self.y_cen; dist_z = pos[:,2]-self.z_cen;
                        dist = np.sqrt(dist_x**2. + dist_y**2. + dist_z**2.)
                        mask = (dist <= r_search)
                        x.extend(pos[mask,0].tolist()) # not append! 
                        y.extend(pos[mask,1].tolist())
                        z.extend(pos[mask,2].tolist())

        return [np.array(x), np.array(y), np.array(z)]    
    
  
    """
    Read in a radial/hpix cell

    filename -- The name of the file to read, or a file object. If file
                object, will not be closed upon function return. Instead
                the pointer will be left at the location of the last
                data read.
    read_xxx -- Whether or not to read xxx

    positions and velocities are flattted
    use: reshape(npart, 3)
    """

    """
    Follow convention of Ken Osato: Use reduced quadropole moment to find axis ratio of ellipsoidal cluster
    1. Project onto principle axes spitted out by quadropole tensor
    2. Do not remove particles. Particles chosen for those inside Rvir
    3. Use Reduced tensor
    4. q, s refer to ratio of minor to major, and intermediate to major axis

    Returns:
    converge -- Boolean
    [a,b,c] -- normalized major, intermediate, minor axes lengths (only ratio matters in reduced tensor)
    [lx, ly, lz] -- direction of minor, intermediate, major
    """
    def quad_moment(self, ptcl_coord):
        
        #Selects particle with elliptical radius within Rmax. The axis direction and ratio are from 
        #the object itself outside the function, different from the axis direction and ratio generated
        #from the quad_moment() function. 
        
        #[a,b,c] -- normalized major, intermediate, minor axes lengths (only ratio matters in reduced tensor)
        #[lx, ly, lz] -- direction of minor, intermediate, major
        a = self.axis_ratio[0]; b = self.axis_ratio[1]; c = self.axis_ratio[2]
        q = c/a; s = b/a
        lx = self.axis_dir[0]; ly=self.axis_dir[1]; lz=self.axis_dir[2]
        
        ptcl_coord_x = ptcl_coord[0]; ptcl_coord_y = ptcl_coord[1]; ptcl_coord_z = ptcl_coord[2]
        rx = ptcl_coord_x - self.x_cen
        ry = ptcl_coord_y - self.y_cen
        rz = ptcl_coord_z - self.z_cen
        r_carte = np.array([rx, ry, rz]).T
        rx_proj = np.dot(r_carte, lx) 
        ry_proj = np.dot(r_carte, ly)
        rz_proj = np.dot(r_carte, lz)   
        R_range = np.sqrt((rx_proj/q)**2. + (ry_proj/s)**2. + rz_proj**2.) #Elliptical radius

        #Choose particles inside elliptical Rvir
        ptcl_range = np.where(R_range < self.Rmax)
        rx = rx[ptcl_range]; ry = ry[ptcl_range]; rz = rz[ptcl_range]

        num_mem_ptcl = len(rx)
        if num_mem_ptcl < 100:
            return 0, False, [1,1,1], np.eye(3)
        #print "Number of particles inside virial radius is ", num_mem_ptcl

        
        
        #Part below same as original (before making into class object). The axis ratio a,b,c
        #and directions lz,ly,lx are different from the one called from the object. 
        #The ratio and direction if converged will become that of the object. 
        
        #Building quadrupole tensor. 
        Rp = np.sqrt(rx**2. + ry**2. + rz**2.)
        r = np.matrix([rx,ry,rz])
        r_rdu = r/Rp
        M_rdu = r_rdu*r_rdu.T #Initial quadrupole tensor before iteration

        #Finding eigvec, eigval
        M_eigval, M_eigvec = np.linalg.eig(M_rdu)
        sort_eigval = np.argsort(M_eigval)[::-1]
        a, b, c = np.sqrt(M_eigval[sort_eigval]/num_mem_ptcl) #a, b, c major, intermediate, minor
        lx, ly, lz = M_eigvec.T[sort_eigval][::-1] #lx, ly, lz minor, intermediate, major (order reversed from a, b, c)
        lx = np.array(lx)[0]; ly = np.array(ly)[0]; lz = np.array(lz)[0]

        #Sanity check
        """
        print "r_rdu", r_rdu
        check_eig = M_rdu.dot(lx) - num_mem_ptcl*c**2.*lx
        print "M_rdu.dot(lx) ", np.dot(np.array(M_rdu), lx)
        print "check_eig ", check_eig
        print "lx is ", lx
        print "M_eigvec.T[sort_eigval], ", M_eigvec.T[sort_eigval]
        print "M_eigvec[:,0] ", M_eigvec[:,0]
        print "M_eigvec[sort_eigval] ", M_eigvec[sort_eigval]
        print "M_eigvec", M_eigvec
        print "sort_eigval ", sort_eigval
        """

        #Initial conditions
        q_prev = 1.; s_prev = 1.
        converge = False
        conv_iter = 0

        P_tot = np.eye(3) #the multiplicative product of all projections done over each iteration
        while (not converge) & (conv_iter < 100):
            #Change of basis
            P_axis = np.matrix([lx,ly,lz])
            P_tot = P_axis*P_tot
            r_proj = P_axis*r
            rx = np.array(r_proj[0,:])[0]; ry = np.array(r_proj[1,:])[0]; rz = np.array(r_proj[2,:])[0]

            #New iteration
            q_cur = c/a; s_cur = b/a #Osato conventaion
            Rp = np.sqrt((rx/q_cur)**2. + (ry/s_cur)**2. + rz**2.)
            r = np.matrix([rx, ry, rz])
            r_rdu = r/Rp
            M_rdu = r_rdu*r_rdu.T
            M_eigval, M_eigvec = np.linalg.eig(M_rdu)
            sort_eigval = np.argsort(M_eigval)[::-1]
            a, b, c = np.sqrt(M_eigval[sort_eigval]/num_mem_ptcl)
            lx, ly, lz = M_eigvec.T[sort_eigval][::-1]
            lx = np.array(lx)[0]; ly = np.array(ly)[0]; lz = np.array(lz)[0]

        #if unit-lengths are too small may generate error. Exit function as not converged
        if any([x < 0.01 for x in [a,b,c]]):
            print("Error: axis_len too small. a,b,c={},{},{}").format(a,b,c)
            return 0, False, [1,1,1], np.eye(3)

	    #test converge
            conv_err = 1e-6
            conv_s = np.abs(1 - s_cur/s_prev); conv_q = np.abs(1 - q_cur/q_prev)
            converge = (conv_s < conv_err) & (conv_q < conv_err)
            conv_iter += 1
            q_prev = q_cur; s_prev = s_cur

        #find lx, ly, lz in original basis
        P_inv = np.linalg.inv(P_tot)
        l_new_basis = np.matrix([lx,ly,lz]).T
        l_orig_basis = np.transpose(P_inv*l_new_basis)
        lx_orig = np.array(l_orig_basis[0])[0]
        ly_orig = np.array(l_orig_basis[1])[0]
        lz_orig = np.array(l_orig_basis[2])[0]


        return num_mem_ptcl, converge, [a,b,c], np.array([lx_orig, ly_orig, lz_orig])

In [ ]:
cluster = redM_data[0]
cluster_id = cluster['cluster_id']
id_cen = cluster['id_cen_0']
ra = cluster['ra']
dec = cluster['dec']
red = cluster['redshift']
# match central cartesian coordinates
#xcen
#ycen
#zcen
R_lambda = cluster['R_lambda']

test_shape = HaloShape(cluster_id = cluster_id
                 , id_cen_0 = id_cen
                 , ra_cen = ra
                 , dec_cen = dec
                 , red_cen = red
                 , x_cen = 100
                 , y_cen = 100
                 , z_cen = 100
                 , Rmax = R_lambda
                )

In [43]:
print('Cluster ID:', test_shape.cluster_id)

Cluster ID: 12
